In [40]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy import Column, Integer, String, Float, ForeignKey,Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
Base = declarative_base() 

In [100]:
dept_path = r"data/departments.csv"
deptemp_path = r"data/dept_emp.csv"
deptman_path = r"data/dept_manager.csv"
emp_path = r"data/employees.csv"
salary_path = r"data/salaries.csv"
title_path = r"data/titles.csv"

In [101]:
dept_df = pd.read_csv(dept_path)
deptemp_df = pd.read_csv(deptemp_path)
deptman_df = pd.read_csv(deptman_path)
emp_df = pd.read_csv(emp_path)
salary_df = pd.read_csv(salary_path)
title_df = pd.read_csv(title_path)


In [43]:
class Title(Base):
    __tablename__ = "titles"
    id =  Column(Integer, primary_key=True)
    title_id = Column(String(255))
    title = Column(String(255))

In [44]:
class Department(Base):
    __tablename__ = "departments"
    id =  Column(Integer, primary_key=True)
    dept_no = Column(String(255))
    dept_name = Column(String(255))

In [45]:
class Employee(Base):
    __tablename__ = "employees"
    id =  Column(Integer, primary_key=True)
    emp_no = Column(String(255))
    emp_title_id = Column(Integer, ForeignKey(Title.title_id),primary_key = True) 
    birth_date = Column(Date)
    first_name = Column(String(255))
    last_name = Column(String(255))
    sex = Column(String(255))
    hire_date = Column(Date)

In [150]:
class Manager(Base):
    __tablename__ = "managers"
    id =  Column(Integer, primary_key=True)
    dept_no = Column(String(255), ForeignKey(Department.dept_no), primary_key = True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)

g:\programs\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Manager, and will be replaced in the string-lookup table.
  """Entry point for launching an IPython kernel.


InvalidRequestError: Table 'managers' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [47]:
class Salary(Base):
    __tablename__ = "salaries"
    id =  Column(Integer, primary_key=True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)
    salary = Column(Integer)

In [48]:
class Department_Employee(Base):
    __tablename__ = "department_employees"
    id =  Column(Integer, primary_key=True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)
    dept_no = Column(String(255), ForeignKey(Department.dept_no), primary_key = True)

In [102]:
def deptf(ind,df):
    return Department(dept_no = df.iloc[ind]["dept_no"], dept_name = df.iloc[ind]["dept_name"])

In [85]:
def titlef(ind,df):
    return Title(title_id = df.iloc[ind]["title_id"], title = df.iloc[ind]["title"])

In [103]:
def deptempf(ind,df):
    return Department_Employee(dept_no = df.iloc[ind]["dept_no"], emp_no = df.iloc[ind]["emp_no"])

In [104]:
def deptmanf(ind,df):
    return Manager(dept_no = df.iloc[ind]["dept_no"], emp_no = df.iloc[ind]["emp_no"])

In [106]:
def salaryf(ind,df):
    return Salary(emp_no = df.iloc[ind]["emp_no"], salary = df.iloc[ind]["salary"])

In [107]:
def empf(ind,df):
    return Employee(emp_no = df.iloc[ind]["emp_no"], 
                    emp_title = df.iloc[ind]["emp_title"],
                    birth_date = df.iloc[ind]["birth_date"],
                    first_name = df.iloc[ind]["first_name"], 
                    last_name = df.iloc[ind]["last_name"],
                    sex = df.iloc[ind]["sex"],
                    hire_date = df.iloc[ind]["hire_date"], )

In [124]:
def add_df_to_table_please(df,fx,session):
    num_rows = len(df.index)
    for i in range(num_rows):
        session.add(fx(i,df))
    session.commit()

In [70]:
def deletef(table,session):
    for i in table:
        session.delete(i)
    session.commit()

In [57]:
engine = create_engine('sqlite:///data/johnson.sqlite')
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [76]:
# inspector = inspect(engine)
# inspector.get_table_names()
# inspector.get_columns('employees')

# that_one = session.query(Department).filter(Department.dept_name == "Marketing").first()
# session.delete(that_one)
# for dpt in deptso:
#     print(dpt.title)
# session.commit()

In [108]:
depts = session.query(Department)
deptemps = session.query(Department_Employee)
deptmans = session.query(Manager)
emps = session.query(Employee)
salaries = session.query(Salary)
titles = session.query(Title)

In [149]:
# add_df_to_table_please(dept_df,deptf,session) check!!
# add_df_to_table_please(deptemp_df,deptempf,session)
add_df_to_table_please(deptman_df,deptmanf,session)
# add_df_to_table_please(emp_df,empf,session)
# add_df_to_table_please(salary_df,salaryf,session)
# add_df_to_table_please(title_df,titlef,session) check!!
# session.rollback()

IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: managers.id
[SQL: INSERT INTO managers (dept_no, emp_no) VALUES (?, ?)]
[parameters: ('d001', 110022)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [142]:
depts_2 = session.query(Department)
deptemps_2 = session.query(Department_Employee)
deptmans_2 = session.query(Manager)
emps_2 = session.query(Employee)
salaries_2 = session.query(Salary)
titles_2 = session.query(Title)
# deletef(depts,session)

In [144]:
for t in deptmans_2:
    print(t.dept_no)

In [148]:
session.rollback()

g:\programs\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  """Entry point for launching an IPython kernel.
